In [4]:
import pandas as pd
import numpy as np
import openpyxl
import sys
import os
from pathlib import Path

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")

from players_season_builder import *

In [5]:
player_path = r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player"
file_type, sheet_name = "xlsx" , 0
data_source = "BDB-PLAYER"

player_season_data = combine_xl_data(player_path, file_type, sheet_name, data_source)

\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2006-2007-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2007-2008-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2008-2009-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2009-2010-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2010-2011-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Big Data - Player\1. Player\NBA-2011-2012-Player-BoxScore-Dataset.xlsx
\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\

In [11]:
player_per_data = pd.read_excel(
    r"\Users\sebas\Desktop\UChicago - Q6\Sports Analytics\sports_analytics_project\data\Player Per Game PER Data.xlsx",
    sheet_name='Agg PER >24mins,NoPlayoffs')

In [12]:
# Game Player
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2020 Playoffs', '2020 Playoffs')
player_season_data['DATASET'] = player_season_data['DATASET'].str.replace(
    'NBA 2019 Playoffs','2019 Playoffs')

player_season_data['DATE'] = pd.to_datetime(
    player_season_data['DATE']
    )

# Player PER Data
player_per_data['DATA SET'] = player_per_data['DATA SET'].str.replace(
    'NBA 2018-2019 Regular Season',
    '2018-2019 Regular Season'
)
player_per_data['DATA SET'] = player_per_data['DATA SET'].str.replace(
    'NBA 2019-2020 Regular Season',
    '2019-2020 Regular Season'
)

# Filter for Regular Season
player_regular_season = player_season_data[player_season_data['DATASET'].str.contains("Season")]
# Filter for playoffs
player_playoff = player_season_data[player_season_data['DATASET'].str.contains("Playoffs")]


KeyError: 'DATASET'

In [10]:
player_season_avg = player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).mean().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_var= player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).var().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_sd = player_regular_season.groupby(['DATASET','PLAYER FULL NAME']).std().sort_values(['DATASET','PLAYER FULL NAME'])
player_season_HR= player_regular_season.groupby(['DATASET','PLAYER FULL NAME','VENUE']).mean().sort_values(['DATASET','PLAYER FULL NAME'])

Based on season averages. Filter players who averaged more than 24 minutes 

This should hopefully make the following calculations faster. 



In [1]:
player_season_avg[player_season_avg['MIN'] >= 24].columns

NameError: name 'player_season_avg' is not defined

In [5]:
# player_regular_season.loc[player_regular_season['DATASET'] == '2018-2019 Regular Season'].sort_values(by=['PLAYER FULL NAME','DATE'])['DATE'].diff()
list(player_regular_season[player_regular_season['DATASET'] == '2018-2019 Regular Season']['PLAYER FULL NAME'].unique())

['Dario Saric',
 'Robert Covington',
 'Joel Embiid',
 'Markelle Fultz',
 'Ben Simmons',
 'Amir Johnson',
 'JJ Redick',
 'T.J. McConnell',
 'Landry Shamet',
 'Jonah Bolden',
 'Furkan Korkmaz',
 'Jayson Tatum',
 'Gordon Hayward',
 'Al Horford',
 'Jaylen Brown',
 'Kyrie Irving',
 'Marcus Morris',
 'Terry Rozier',
 'Marcus Smart',
 'Aron Baynes',
 'Daniel Theis',
 'Semi Ojeleye',
 'Guerschon Yabusele',
 'Brad Wanamaker',
 'Paul George',
 'Patrick Patterson',
 'Steven Adams',
 'Terrance Ferguson',
 'Dennis Schroder',
 'Jerami Grant',
 'Alex Abrines',
 'Raymond Felton',
 'Nerlens Noel',
 'Hamidou Diallo',
 'Kevin Durant',
 'Draymond Green',
 'Damian Jones',
 'Klay Thompson',
 'Stephen Curry',
 'Kevon Looney',
 'Andre Iguodala',
 'Quinn Cook',
 'Shaun Livingston',
 'Jordan Bell',
 'Alfonzo McKinnie',
 'Jonas Jerebko',
 'Khris Middleton',
 'Giannis Antetokounmpo',
 'Brook Lopez',
 'Malcolm Brogdon',
 'Eric Bledsoe',
 'Donte DiVincenzo',
 'Ersan Ilyasova',
 'John Henson',
 'Tony Snell',
 'Chris

In [6]:
test = list()
for season in player_regular_season['DATASET'].unique():
    for player in list(player_regular_season[player_regular_season['DATASET'] == season]['PLAYER FULL NAME'].unique()):
        temp_season = player_regular_season.loc[
            player_regular_season['DATASET'].eq(season) & player_regular_season['PLAYER FULL NAME'].eq(player)].sort_values(by=['PLAYER FULL NAME', 'DATE'])
        temp_season['DATE-DIFF'] = temp_season['DATE'].diff().dt.days
        test.append(temp_season)

player_regular_season = pd.concat(test)
player_regular_season['DATE-DIFF'] = player_regular_season['DATE-DIFF'].astype(str)
player_regular_season['DATE-DIFF'] = player_regular_season['DATE-DIFF']+" days"

In [7]:
player_regular_season_dummies = pd.concat([
    player_regular_season,
    pd.get_dummies(player_regular_season['DATE-DIFF'])
], axis=1)

In [10]:
with pd.ExcelWriter("../data/BDB_Player.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    player_season_data.to_excel(writer, sheet_name="General RS", index=False)
    # player_season_avg.to_excel(writer, sheet_name="RS - Avg", index=True)
    # player_season_var.to_excel(writer, sheet_name= "RS - Var", index=True)
    # player_season_sd.to_excel(writer, sheet_name="RS - SD", index=True)
    player_regular_season_dummies.to_excel(writer, sheet_name="RS - Dummies", index=False)
    player_season_HR.to_excel(writer, sheet_name="RS - HomeAway", index=True)